In [1]:
## Initializations
from config import login, password, API_KEY
from client_demo import *
from tradingscrips import *
from webhook_simulatior import *
import visualizations
import time



In [4]:
## Loop through time
while True:
    time.sleep(2) #sleep for x seconds
    # in gitignore
    cl = Client(
        login,
        password,
        API_KEY
    )
    # Get Information about all subaccounts
    subaccounts = cl.all_accounts()
    data = json.loads(subaccounts) 
    subaccount_names = [account["accountName"] for account in data["accounts"]]
    subaccount_balance = [account["balance"]["available"] for account in data["accounts"]]
    subaccount_profitloss = [account["balance"]["profitLoss"] for account in data["accounts"]]
    subaccount_IDs = [account["accountId"] for account in data["accounts"]]

    # Tickers of interest are found in the watchlist 
    # !!(Maybe update the Watchlist ID here / create multiple)
    watchlists = cl.watchlist_stockwatch()
    data = json.loads(watchlists)
    tickers = [entry['epic'] for entry in data['markets']]

    # delete all tickers with open positions 
    positions_open = []
    all_positions_tickers = []
    for account in subaccount_IDs:
        cl.switch_account(account)
        plh = json.loads(cl.all_positions())
        if plh.get("positions"):
            positions_open.extend([entry['market']['epic'] for entry in plh['positions']])
            #all_positions_tickers = [entry['market']['epic'] for entry in positions_open['positions']]

    positions_dict = {} #{'Demo_1': ['ETHUSD', 'BTCUSD', 'XLMUSD'], 'Demo_2': ['AIRFR']}
    for index in range(0,len(subaccount_names)):
        cl.switch_account(subaccount_IDs[index])
        plh = json.loads(cl.all_positions())
        if plh.get("positions"):
            direction = [entry ['position']['direction'] for entry in plh['positions']]
            positions = [entry['market']['epic'] for entry in plh['positions']]
            dealIDs = [entry['position']['dealId'] for entry in plh['positions']]
            positions_dict[subaccount_names[index]] = (positions, direction, dealIDs)

    # Remove elements from A that are also in B#
    unique_positions = []
    if positions_open:
        unique_positions = list(set(positions_open)) 
        #tickers = [x for x in tickers if x not in unique_positions]

    # Make trading decisions
    for ticker in tickers:
        epic = ticker
        resolution = ResolutionType.MINUTE_30
        max = 300
        # get prices
        prices = cl.historical_prices(epic,resolution, max)
        data = json.loads(prices)
        snapshot_times = [entry['snapshotTime'] for entry in data['prices']]
        bid_prices = [entry['openPrice']['bid'] for entry in data['prices']]
        ask_prices = [entry['openPrice']['ask'] for entry in data['prices']]
        high_prices = [entry['highPrice']['ask'] for entry in data['prices']]
        low_prices = [entry['lowPrice']['ask'] for entry in data['prices']]
        open_prices = [entry['openPrice']['ask'] for entry in data['prices']]
        close_prices = [entry['closePrice']['ask'] for entry in data['prices']]
        prices = {'ticker':ticker,
                'bid':bid_prices,
                'ask': ask_prices,
                'high': high_prices,
                'low': low_prices,
                'open': open_prices,
                'close': close_prices
                }
        
        ######################
        # Make actions
        # ACCOUNT 1:
        #action = Scripts.simpletest(prices, positions_dict['Demo_1'])
        if 'Demo_1' not in positions_dict: positions_dict['Demo_1'] = []
        action = Scripts.AllAlerts(prices, positions_dict['Demo_1'], subaccount_balance[0])
        account = "Demo_1"
        restart = False
        if action == 'BUY' or action == 'SELL' or action == 'CLOSE':
            if webhook(epic,action,prices,account, positions_dict) == 'ACCEPTED':
                visualizations.write_to_csv(action, ticker, account) # update log file
                print('action', account, action, ticker)
                restart = True
        if restart == True:
            break
        if restart == False:
            # prints out 'none' actions
            print('no action', account, action, ticker)
    if restart == True:
        break
#################### hier geht es weiter mit den nächsten Subaccounts und Scripts #######################
        #### ACCOUNT 2:
        ###action = Scripts.GPT_intuitive(bid_prices, positions_dict['Demo_2']) #bullshit, zehn Mal die gleiche action dann 

        

no action Demo_1 Nix machen TSLA
no action Demo_1 Nix machen PYPL
no action Demo_1 SELL ENRUS
no action Demo_1 Nix machen AIRFR
no action Demo_1 BUY CAT
no action Demo_1 SELL BYND
no action Demo_1 BUY ARKK
no action Demo_1 BUY TEAM
no action Demo_1 CLOSE ADSGN
no action Demo_1 SELL CTLT
no action Demo_1 CLOSE GOOGL
no action Demo_1 Nix machen V
no action Demo_1 Nix machen NVDA
no action Demo_1 CLOSE AAPL
no action Demo_1 Nix machen TSLA
no action Demo_1 Nix machen PYPL
no action Demo_1 SELL ENRUS
no action Demo_1 Nix machen AIRFR
no action Demo_1 BUY CAT
no action Demo_1 SELL BYND
no action Demo_1 BUY ARKK
no action Demo_1 BUY TEAM


KeyboardInterrupt: 

In [ ]:
#####
# implement second algorithm
# run on raspi continuously
# improve prints upon actions
# extend excel documentation
# use graphs for developing new algorithms
# develop new section for backtesting strategies
# terminate contract with trading view and Amazon AWS
##################
# bring code to presentable state for collaborators # Max, Enrico, Max, Yongwei
# dont fetch whole data every time - just add the missing one 